# Final Resource

final resource as a RMarkdown webpage

- The purpose of your topic is clearly captured in your resource
- Quantitative questions are clearly and concisely explained with thoughtful text and compelling visuals
- A nuanced understanding of the important features of the dataset and topic is demonstrated.
- High-level insights (important descriptive information, major trends, notable outliers, etc.) should be prominent in your resource.
- Methods and results of statistical approaches are clear
- Your analysis should be easily reproducible using information in your GitHub repository
- Remember to tailor your resource to a specific target audience. The amount of framing you need to do for a scientific versus general audience is quite different